<a href="https://colab.research.google.com/github/mahopman/IEBM-Net/blob/main/pretraining_dataset/tag_drug_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
__author__ = 'Qiao Jin'
__editor__ = 'Mia Hopman'

In [3]:
local_path = '/content/drive/MyDrive/MS_DataScience/DS595/IEBM-Net_Data'
pretraining_dataset_path = f'{local_path}/pretraining_dataset'

In [4]:
!wget http://nlp.stanford.edu/software/stanford-postagger-full-2015-04-20.zip -P {local_path}/stanford_pos
!unzip {local_path}/stanford_pos/stanford-postagger-full-2015-04-20.zip -d {local_path}/stanford_pos

--2024-05-05 18:27:53--  http://nlp.stanford.edu/software/stanford-postagger-full-2015-04-20.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-postagger-full-2015-04-20.zip [following]
--2024-05-05 18:27:54--  https://nlp.stanford.edu/software/stanford-postagger-full-2015-04-20.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-postagger-full-2015-04-20.zip [following]
--2024-05-05 18:27:54--  https://downloads.cs.stanford.edu/nlp/software/stanford-postagger-full-2015-04-20.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:4

In [7]:
!git clone https://github.com/Alibaba-NLP/EBM-Net.git
!cp /content/EBM-Net/pretraining_dataset/sec2label.json -d {pretraining_dataset_path}/sec2label.json

fatal: destination path 'EBM-Net' already exists and is not an empty directory.


In [8]:
def mask_and_label(sent):
	sent = ' ' + sent
	lower_sent = sent.lower()

	if ' than ' in lower_sent and all([exc not in lower_sent for exc in exclude]):
		words = tokenize.word_tokenize(sent)
		lowers = [word.lower() for word in words]
		words_ctr = Counter(lowers)

		if words_ctr['than'] == 1: # more than 1 are not useful (mostly describing only quantitative relations)
			than_idx = lowers.index('than')
			inter = set(lowers[:than_idx]).intersection(key_words)

			if len(inter) >= 1:
				up_indices = [1 if word.lower() in ups else 0 for word in words]
				down_indices = [1 if word.lower() in downs else 0 for word in words]

				if any(up_indices) and not any(down_indices):
					if than_idx + 1 < len(lowers) and (lowers[than_idx+1].isnumeric() or lowers[than_idx+1] in nums):
						pass
					else:
						indices = [idx_ for idx_, up in enumerate(up_indices) if up == 1] + [than_idx]
						final = words
						direction = 2

				elif any(down_indices) and not any(up_indices):
					if than_idx + 1 < len(lowers) and (lowers[than_idx+1].isnumeric() or lowers[than_idx+1] in nums):
						pass
					else:
						indices = [idx_ for idx_, down in enumerate(down_indices) if down == 1] + [than_idx]
						final = words
						direction = 0

	elif ' similar' in lower_sent and ' to ' in lower_sent:
		words = tokenize.word_tokenize(sent)
		lowers = [word.lower() for word in words]
		words_ctr = Counter(lowers)

		for idx, lower in enumerate(lowers):
			if lower in sims:
				sim_idx = idx
				break

		if 'sim_idx' in locals():
			if 'to' in lowers[sim_idx:]:
				to_idx = sim_idx + lowers[sim_idx:].index('to')

				indices = [sim_idx] + [to_idx]
				final = words
				direction = 1

	elif ' no' in lower_sent and ' differ' in lower_sent and 'and' in lower_sent:
		words = tokenize.word_tokenize(sent)
		lowers = [word.lower() for word in words]
		words_ctr = Counter(lowers)

		for idx, word in enumerate(lowers):
			if word in diffs:
				diff_idx = idx
				break

		if 'diff_idx' in locals():
			# first find the left no, then scan the middle words
			for i in range(idx):
				word = words[idx-1-i]
				if word in nos:
					no_idx = idx-1-i
					break

			if 'no_idx' in locals():
				bet_indices = [1 if word == 'between' and idx > diff_idx else 0 for idx, word in enumerate(lowers)]

				if any(bet_indices):
					first_bet = bet_indices.index(1)
					if 'and' in lowers[first_bet:]:
						and_idx = first_bet + lowers[first_bet:].index('and')
						indices = list(range(no_idx, diff_idx+1)) + [idx for idx, bet in enumerate(bet_indices) if bet == 1] + [and_idx]
						final = words
						direction = 1

	if 'final' in locals() and 'direction' in locals():
		if type(final) == list and len(final) > 0 and final[-1] != '?':
			return [final, direction, indices]

	else:
		return False

In [9]:
def process(item):
	# an item is an article
	# also need to save the context
	# as well as save the evidence
	pmid = item['pmid']
	texts = item['texts']
	labels = item['sec_labels']

	evi_output = []
	ctx_output = {'pmid': pmid, 'ctx': ''}

	bg_status = True

	for text, label in zip(texts, labels):
		if label == 'TITLE': continue
		sents = tokenize.sent_tokenize(text)
		if not label or label not in sec2label:
			for sent in sents:
				result = mask_and_label(sent)
				if result:
					bg_status = False
					evi_output.append({'pmid': pmid, 'pos': result[0], 'label': result[1], 'indices': result[2]})
				else:
					if bg_status:
						ctx_output['ctx'] += ' ' + sent
		else:
			judge = sec2label[label]

			if judge == '1': # all background
				ctx_output['ctx'] += ' ' + text
			else:
				bg_status = False # starting no background
				for sent in sents:
					result = mask_and_label(sent)
					if result: evi_output.append({'pmid': pmid, 'pos': result[0], 'label': result[1], 'indices': result[2]})

	return evi_output, ctx_output

In [11]:
!mkdir {pretraining_dataset_path}/evidence

In [ ]:
from collections import Counter

import glob
import json
import os
import nltk
from nltk import tokenize
from nltk.tag import StanfordPOSTagger
from nltk.corpus import stopwords

import sys

import random as rd

nltk.download('punkt')

'''
used to pseudo label the dataset
'''

jar = f'{local_path}/stanford_pos/stanford-postagger-full-2015-04-20/stanford-postagger.jar'
model = f'{local_path}/stanford_pos/stanford-postagger-full-2015-04-20/models/english-left3words-distsim.tagger'

st = StanfordPOSTagger(model, jar, encoding='utf8')

exclude = set(['rather than', 'other than'])
ups = set(['better', 'greater', 'higher', 'later', 'more', 'faster', 'older', 'longer', \
		'larger', 'broader', 'wider', 'stronger', 'deeper', 'more', 'commoner', 'richer', \
		'further', 'bigger'])
downs = set(['worse', 'smaller', 'lower', 'earlier', 'less', 'slower', 'younger', 'shorter', \
		'smaller', 'narrower', 'narrower', 'weaker', 'shallower', 'fewer', 'rarer', 'poorer', \
		'closer', 'smaller'])

key_words = ups.union(downs)

diffs = set(['difference', 'differences', 'different', 'differently', 'differ'])
sims = set(['similar', 'similarly', 'similarity', 'similarities'])

nos = set(['no', 'not'])
middles = set(['significant', 'significantly', 'statistic', 'statistically', 'statistical'])

nums = set(["twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety", "zero", \
            "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten", "eleven", \
            "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen"])

sec2label = json.load(open(f'{pretraining_dataset_path}/sec2label.json'))

files = glob.glob(f'{pretraining_dataset_path}/pubmed_baseline/pubmed24n*.json')
for file in files:
    chunk_id = file.split('/')[-1].split('.')[0].split('n')[-1]
    if not os.path.exists(f'{pretraining_dataset_path}/evidence/evidence_pos_{chunk_id}.json'):
        evi_output = []
        ctx_output = []

        data = json.load(open(file))

        for item in data:
            results = process(item)
            evi_output += results[0]
            ctx_output.append(results[1])

        pos_list = st.tag_sents(o['pos'] for o in evi_output)

        for _idx in range(len(evi_output)):
            evi_output[_idx]['pos'] = pos_list[_idx]

        with open(f'{pretraining_dataset_path}/evidence/evidence_pos_{chunk_id}.json', 'w') as f:
            json.dump(evi_output, f)
        with open(f'{pretraining_dataset_path}/evidence/contexts_{chunk_id}.json', 'w') as f:
            json.dump(ctx_output, f)

    else:
        evi_output = json.load(open(f'{pretraining_dataset_path}/evidence/evidence_pos_{chunk_id}.json'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
